In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
#rdd = sc.textFile('book.txt')
rdd0=  sc.parallelize(list(range(10)))

In [3]:
rdd0.top(5)

[9, 8, 7, 6, 5]

In [5]:
rdd0.collect()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [6]:
rdd = sc.textFile('book.txt')
rdd.first()

'The Project Gutenberg EBook of English Coins and Tokens, by '

In [7]:
rdd.map(lambda x : x.split()) \
    .take(3)

[['The',
  'Project',
  'Gutenberg',
  'EBook',
  'of',
  'English',
  'Coins',
  'and',
  'Tokens,',
  'by'],
 ['Llewellynn', 'Jewitt', 'and', 'Barclay', 'V.', 'Head'],
 []]

In [8]:
rdd.flatMap(lambda x : x.split()) \
    .take(3)

['The', 'Project', 'Gutenberg']

In [33]:
from heapq import nlargest
def mapper(_, rows):
    for row in rows:
        for word in row.split():
            yield word



rdd.mapPartitionsWithIndex(mapper) \
    .map(lambda x: (x,1)) \
    .groupByKey() \
    .mapValues(lambda x: sum(x)) \
    .map(lambda x: ('bronx',x)) \
    .groupByKey() \
    .flatMap(lambda g: (g[0],nlargest(3,g[1],key=lambda e:e[1]))) \
    .collect()



#rdd.mapPartitionsWithIndex(mapper) \
#    .map(lambda x: (x,1)) \
#    .groupByKey() \
#    .mapValues(lambda x: sum(x)) \
#    .collect()

#rdd.mapPartitionsWithIndex(mapper) \
#    .map(lambda x: (x,1)) \
#    .reduceByKey(lambda x, y : x+y) \
#    .saveAsTextFile("output")

['bronx', [('the', 2104), ('of', 1631), ('and', 1277)]]

In [26]:
SAT_FN='SAT_Results.csv'
HSD_FN='DOE_High_School_Directory_2014-2015.csv'

In [52]:
sat = sc.textFile(SAT_FN).cache()

In [53]:
list(enumerate(sat.first().split(',')))

[(0, 'DBN'),
 (1, 'SCHOOL NAME'),
 (2, 'Num of SAT Test Takers'),
 (3, 'SAT Critical Reading Avg. Score'),
 (4, 'SAT Math Avg. Score'),
 (5, 'SAT Writing Avg. Score')]

In [71]:
def extractScore(pid, rows):
    if pid == 0:
        next(rows)
    import csv
    for fields in csv.reader(rows):
        if fields[2] != 's':
            yield (fields[0], (int(fields[2]), int(fields[4])))
    
    
satScore =sat.mapPartitionsWithIndex(extractScore)
satScore.take(4)

[('02M047', (16, 400)),
 ('21K410', (475, 437)),
 ('30Q301', (98, 440)),
 ('17K382', (59, 374))]

In [72]:
schools = sc.textFile(HSD_FN).cache()
list(enumerate(schools.first().split(',')))

[(0, 'dbn'),
 (1, 'school_name'),
 (2, 'boro'),
 (3, 'building_code'),
 (4, 'phone_number'),
 (5, 'fax_number'),
 (6, 'grade_span_min'),
 (7, 'grade_span_max'),
 (8, 'expgrade_span_min'),
 (9, 'expgrade_span_max'),
 (10, 'bus'),
 (11, 'subway'),
 (12, 'primary_address_line_1'),
 (13, 'city'),
 (14, 'state_code'),
 (15, 'zip'),
 (16, 'website'),
 (17, 'total_students'),
 (18, 'campus_name'),
 (19, 'school_type'),
 (20, 'overview_paragraph'),
 (21, 'program_highlights'),
 (22, 'language_classes'),
 (23, 'advancedplacement_courses'),
 (24, 'online_ap_courses'),
 (25, 'online_language_courses'),
 (26, 'extracurricular_activities'),
 (27, 'psal_sports_boys'),
 (28, 'psal_sports_girls'),
 (29, 'psal_sports_coed'),
 (30, 'school_sports'),
 (31, 'partner_cbo'),
 (32, 'partner_hospital'),
 (33, 'partner_highered'),
 (34, 'partner_cultural'),
 (35, 'partner_nonprofit'),
 (36, 'partner_corporate'),
 (37, 'partner_financial'),
 (38, 'partner_other'),
 (39, 'addtl_info1'),
 (40, 'addtl_info2'),
 (4

In [73]:
def extractSchool(pid, rows):
    if pid == 0:
        next(rows)
    
    import csv
    for fields in csv.reader(rows):
        if len(fields) == 58 and fields[17].isdigit():
            if int(fields[17]) > 500: 
                yield (fields[0], fields[2], int(fields[17]))
    
    
largeSchool =schools.mapPartitionsWithIndex(extractSchool)
largeSchool.take(4)

[('01M450', 'Manhattan', 649),
 ('01M539', 'Manhattan', 1725),
 ('01M696', 'Manhattan', 560),
 ('02M374', 'Manhattan', 548)]

In [84]:
# 
largeSchool.join(satScore) \
    .values() \
    .mapValues(lambda x: (x[0], ( x[0]*x[1]))) \
    .reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1])) \
    .mapValues(lambda x: x[1]//x[0]) \
    .take(5)

[('Bronx', 470),
 ('Staten Island', 477),
 ('Manhattan', 514),
 ('Brooklyn', 487),
 ('Queens', 474)]

In [94]:
dfScores = spark.read.load(SAT_FN, format='csv', header = True, inferSchema=True)
dfScores =dfScores.select('DBN', dfScores['Num of SAT Test Takers'].cast('int').alias('ntakers'),
                          dfScores['`SAT Math Avg. Score`'].cast('int').alias('score')).na.drop()


dfScores = dfScores.select('DBN', 'ntakers', (dfScores.score*dfScores.ntakers).alias('total'))

dfScores.show(10)

+------+-------+------+
|   DBN|ntakers| total|
+------+-------+------+
|02M047|     16|  6400|
|21K410|    475|207575|
|30Q301|     98| 43120|
|17K382|     59| 22066|
|18K637|     35| 13335|
|32K403|     50| 18300|
|09X365|     54| 18306|
|11X270|     56| 22064|
|05M367|     33| 12078|
|14K404|     68| 24276|
+------+-------+------+
only showing top 10 rows



In [98]:
dfSchools = spark.read.load(HSD_FN, format='csv', header = True, inferSchema=True)

dfSchools = dfSchools.na.drop(subset=['boro'])
dfSchools = dfSchools.filter(dfSchools['total_students']> 500)
dfSchools = dfSchools.select('dbn', 'boro')

dfSchools.show()

+------+---------+
|   dbn|     boro|
+------+---------+
|01M450|Manhattan|
|01M539|Manhattan|
|01M696|Manhattan|
|02M374|Manhattan|
|02M400|Manhattan|
|02M408|Manhattan|
|02M412|Manhattan|
|02M413|Manhattan|
|02M416|Manhattan|
|02M418|Manhattan|
|02M420|Manhattan|
|02M425|Manhattan|
|02M475|Manhattan|
|02M489|Manhattan|
|02M519|Manhattan|
|02M520|Manhattan|
|02M529|Manhattan|
|02M542|Manhattan|
|02M580|Manhattan|
|02M600|Manhattan|
+------+---------+
only showing top 20 rows



In [109]:
dfResults = dfSchools.join(dfScores, dfSchools.dbn==dfScores.DBN, how='inner')

dfResults = dfResults.groupBy('boro').sum('ntakers', 'total')
dfResults = dfResults.withColumn('avg', dfResults[2]/dfResults[1]) \
            .select('boro', 'avg')

dfResults.show()

+-------------+------------------+
|         boro|               avg|
+-------------+------------------+
|       Queens| 474.3679400475233|
|     Brooklyn|487.46256168204246|
|Staten Island| 477.9099864130435|
|    Manhattan| 514.9312780989081|
|        Bronx|  470.198606271777|
+-------------+------------------+

